In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../../'))

import numpy as np
from matplotlib import pyplot as plt
import arrayfire as af

from dg_maxwell import params
from dg_maxwell import wave_equation as w1d
from dg_maxwell import utils

af.set_backend('cpu')

/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1193: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)
/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1220: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


In [2]:
u_n = params.u_init

L_p_minus1   = params.lagrange_basis_value[:, 0]
L_p_1        = params.lagrange_basis_value[:, -1]
f_i          = w1d.lax_friedrichs_flux(u_n)
print(f_i)
f_iminus1    = af.shift(f_i, 0, 1)
surface_term = af.blas.matmul(L_p_1, f_i) - af.blas.matmul(L_p_minus1,
                                                           f_iminus1)


arrayfire.Array()
Type: double

[1 10 1 1]
    0.0000     0.0001     0.0183     0.3679     1.0000     0.3679     0.0183     0.0001     0.0000     0.0000 



In [3]:
u = params.u_init
b = w1d.b_vector(u)
A_inverse = af.tile(af.inverse(w1d.A_matrix()), d0 = 1, d1 = 1, d2 = params.N_Elements)

In [4]:
b[0, 0] = 0.
b[-1, -1] = 0.
print(b)

arrayfire.Array()
Type: double

[8 10 1 1]
    0.0000    -0.0000    -0.0000    -0.0013    -0.0131     0.0000     0.0131     0.0013     0.0000     0.0000 
    0.0000    -0.0000    -0.0001    -0.0096    -0.0821     0.0135     0.0723     0.0061     0.0001     0.0000 
   -0.0000    -0.0000    -0.0004    -0.0244    -0.1441     0.0668     0.0964     0.0058     0.0000     0.0000 
   -0.0000    -0.0000    -0.0012    -0.0504    -0.1730     0.1395     0.0821     0.0032     0.0000     0.0000 
   -0.0000    -0.0000    -0.0032    -0.0821    -0.1395     0.1730     0.0504     0.0012     0.0000     0.0000 
   -0.0000    -0.0000    -0.0058    -0.0964    -0.0668     0.1441     0.0244     0.0004     0.0000     0.0000 
   -0.0000    -0.0001    -0.0061    -0.0723    -0.0135     0.0821     0.0096     0.0001     0.0000    -0.0000 
   -0.0000    -0.0000    -0.0013    -0.0131    -0.0000     0.0131     0.0013     0.0000     0.0000     0.0000 




In [5]:
# Setting DBC for the first element
I = af.constant(0., d0 = 1, d1 = params.N_LGL, dtype = af.Dtype.f64)
I[0, 0] = 1
A_inverse[0, :, 0] = I

# Setting DBC for the last element
I = af.constant(0., d0 = 1, d1 = params.N_LGL, dtype = af.Dtype.f64)
I[0, -1] = 1

A_inverse[-1, :, -1] = I

# print(A_inverse)

In [6]:
print(af.reorder(utils.matmul_3D(A_inverse, af.reorder(b, d0 = 0, d1 = 2, d2 = 1)), d0 = 0, d1 = 2, d2 = 1))

arrayfire.Array()
Type: double

[8 10 1 1]
    0.0000    -0.0000    -0.0037    -0.3664    -3.6786     0.0001     3.6787     0.3663     0.0037     0.0000 
    0.0000    -0.0000    -0.0053    -0.4563    -3.8981     0.6386     3.4294     0.2913     0.0026     0.0000 
   -0.0000    -0.0000    -0.0113    -0.7139    -4.2243     1.9582     2.8246     0.1711     0.0011     0.0000 
   -0.0000    -0.0001    -0.0295    -1.2221    -4.1950     3.3814     1.9912     0.0772     0.0003     0.0000 
   -0.0000    -0.0003    -0.0772    -1.9912    -3.3814     4.1950     1.2221     0.0295     0.0001     0.0000 
   -0.0000    -0.0011    -0.1711    -2.8246    -1.9582     4.2243     0.7139     0.0113     0.0000     0.0000 
   -0.0000    -0.0026    -0.2913    -3.4294    -0.6386     3.8981     0.4563     0.0053     0.0000    -0.0000 
   -0.0000    -0.0037    -0.3663    -3.6787    -0.0001     3.6786     0.3664     0.0037     0.0000     0.0000 




In [5]:
u = params.u_init.copy()
print(u)

u[0, 0] = 1
u[-1, -1] = 1
print(u)

arrayfire.Array()
Type: double

[8 10 1 1]
    0.0000     0.0000     0.0001     0.0183     0.3679     1.0000     0.3679     0.0183     0.0001     0.0000 
    0.0000     0.0000     0.0002     0.0236     0.4165     0.9959     0.3223     0.0141     0.0001     0.0000 
    0.0000     0.0000     0.0004     0.0398     0.5308     0.9592     0.2346     0.0078     0.0000     0.0000 
    0.0000     0.0000     0.0011     0.0762     0.6938     0.8553     0.1427     0.0032     0.0000     0.0000 
    0.0000     0.0000     0.0032     0.1427     0.8553     0.6938     0.0762     0.0011     0.0000     0.0000 
    0.0000     0.0000     0.0078     0.2346     0.9592     0.5308     0.0398     0.0004     0.0000     0.0000 
    0.0000     0.0001     0.0141     0.3223     0.9959     0.4165     0.0236     0.0002     0.0000     0.0000 
    0.0000     0.0001     0.0183     0.3679     1.0000     0.3679     0.0183     0.0001     0.0000     0.0000 


arrayfire.Array()
Type: double

[8 10 1 1]
    1.0000     0.0000   